<a href="https://colab.research.google.com/github/uddhavp22/Mamba-BCI-Journal-of-Neural-Engineering/blob/main/MambaJambaNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install mne
import mne
import numpy as np

import torch
import torch.nn as nn

from torch import tensor
mne.set_log_level('ERROR')


import scipy as sp
!pip install torch_geometric
import torch_geometric
from torch_geometric import utils
from torch_geometric.utils import dense_to_sparse
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
import torchvision
import torchvision.transforms as transforms

!pip install mamba-ssm
import torch
from mamba_ssm import Mamba

import warnings

import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from mamba_ssm.modules.block import Block
from functools import partial

# Mamba Jamba Architecture

In [9]:

class Mambablock(nn.Module):
    def __init__(self, d_model, n_blocks, d_state=16, expand=2):
        super(Mambablock, self).__init__()
        self.n_blocks = n_blocks
        self.d_model = d_model
        config = {'d_state': d_state, 'expand': expand}

        # Create Mamba instance with d_model as a keyword argument
        self.four_mamba_blocks = nn.ModuleList([Block(d_model, partial(Mamba,**config), nn.Identity) for _ in range(self.n_blocks)])

    def forward(self, x):
        count=0
        for block in self.four_mamba_blocks:
            x = block(x)[0]
            print(count)
        return x


input_tensor = torch.randn(1, 1001, 1).to("cuda")


#num_channels = 22  # Define num_channels
#num_layers = 4  # Define num_layers, choose as needed
#im = 1 # Define dim, choose as needed
model = Mambablock(d_model=1, n_blocks=4).to("cuda")
print(model(input_tensor).shape)

0
0
0
0
torch.Size([1, 1001, 1])


In [ ]:
class Mambablock(nn.Module):
    def __init__(self, d_model, n_blocks, d_state=16, expand=2):
        super(Mambablock, self).__init__()
        self.n_blocks = n_blocks
        self.d_model = d_model
        config = {'d_state': d_state, 'expand': expand}

        # Create Mamba instance with d_model as a keyword argument
        self.mamba_blocks = nn.ModuleList([Block(d_model, partial(Mamba,**config), nn.Identity) for _ in range(self.n_blocks)])

    def forward(self, x):
        for block in self.mamba_blocks:
            x = block(x)[0]
        return x

class EEGMamba(nn.Module):
    def __init__(self, num_channels, num_blocks, dim, d_state=256, d_conv=4, expand=2,d_model=1):
        super(EEGMamba, self).__init__()
        # Pass dim as d_model to Mambablock
        self.channel_models = nn.ModuleList([Mambablock(dim, num_blocks, d_state, expand) for _ in range(num_channels)])
        #self.linear = nn.Linear(1001, d_state) if u want a linear transform after data

    def forward(self, x):
        batch_size = x.size(0)
        channel_outputs = []

        # Iterate over each channel model and corresponding channel data
        for i in range(x.size(1)):
            channel_data = x[:, i, :, :]  # Extract data for the i-th channel


            model = self.channel_models[i]
            output = model(channel_data)
            averaged_output = torch.mean(output, dim=1, keepdim=True)
            channel_outputs.append(averaged_output)

        # Stack the channel outputs along the correct dimension
        combined_output = torch.stack(channel_outputs, dim=1)
        return combined_output

class MambaJambaNet(nn.Module):
    def __init__(self, num_channels, num_layers, dim, d_state=16, d_conv=4, expand=2, kernel_size=2, num_classes=1):
        super(MambaJambaNet, self).__init__()
        self.mamba_model = EEGMamba(num_channels, num_layers, dim, d_state, d_conv, expand)
        self.classifier = nn.Sequential(
            nn.Linear(num_channels, 46),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(46, num_classes)
        )

    def forward(self, x):
        x = self.mamba_model(x)
        print(x)
        x = x.squeeze(2)  # making it (batch_size, num_channels, dim)
        x = x.squeeze(2)
        x = self.classifier(x)
        return x

input_tensor = torch.randn(1, 22, 1001, 1).to("cuda")
num_channels = 22  # Define num_channels
num_layers = 4  # Define num_layers, choose as needed
dim = 1 # Define dim, choose as needed
model = MambaJambaNet(num_channels=22, num_layers=4, dim=1).to("cuda")
print(model(input_tensor).shape)

#rn total parameters - 11,888

NameError: name 'nn' is not defined

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")


Number of parameters: 11888


# Mamba2d

In [7]:
class Mambablock(nn.Module):
    def __init__(self, d_model, n_blocks, d_state=16, expand=2):
        super(Mambablock, self).__init__()
        self.n_blocks = n_blocks
        self.d_model = d_model
        config = {'d_state': d_state, 'expand': expand}

        # Create Mamba instance with d_model as a keyword argument
        self.four_mamba_blocks = nn.ModuleList([Block(d_model, partial(Mamba,**config), nn.Identity) for _ in range(self.n_blocks)])

    def forward(self, x):
        count=0
        for block in self.four_mamba_blocks:
            x = block(x)[0]
            print(count)
        return x


input_tensor = torch.randn(1, 22, 1001).to("cuda")
input_tensor = input_tensor.permute(0, 2, 1)

#num_channels = 22  # Define num_channels
#num_layers = 4  # Define num_layers, choose as needed
#im = 1 # Define dim, choose as needed
model = Mambablock(d_model=22, n_blocks=8).to("cuda")
print(model(input_tensor).shape)

0
0
0
0
torch.Size([1, 1001, 22])
